In [1]:
#to check if any library already exists in the current env and installs if not.

import importlib

def check_installed(library_name):
  if importlib.util.find_spec(library_name) is None:
      print(f"{library_name} is not installed.")
      !pip install {library_name}
  else:
      print(f"{library_name} is installed.")




In [2]:
libs_to_check = ['transformers','pandas','kaggle','datasets','sentencepiece']

for library in libs_to_check:
  check_installed(library)

transformers is not installed.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 111.2 MB/s eta 0:00:00
pandas is installed.
kaggle is installed.
datasets is not installed.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 

In [5]:
#Run this cell after you upload to the colab notebook directory kaggle.json manually and import kaggle throws an error

!sudo mv /content/drive/MyDrive/ColabNotebooks/kaggle.json /root/.kaggle/

In [6]:
#libraries
import pandas as pd
import kaggle
import zipfile
import transformers
from datasets import Dataset,DatasetDict

In [7]:
creds = '{"username":"your_username","key":"your_api_key"}'

In [8]:
from pathlib import Path

cred_path = Path('~/.kaggle/kaggle.json').expanduser() #expanduser uses "~" to expand user's root directory


if not cred_path.exists():
  print("File doesnt exist")
  cred_path.parent.mkdir(exist_ok = True) #if the file doesnt exist creates a parent directory

  cred_path.write_text(creds) #This line writes the contents of the creds variable to the file represented
                              # by the cred_path Path object. The write_text() method is a method of the Path object that writes a string to a file.
  cred_path.chmod(0o600)
  #This line sets the file permissions of the file represented by the cred_path Path object to 0o600, 
  #which means that only the owner of the file can read or write it. 
  #The chmod() method is a method of the Path object that sets the file permissions.

In [9]:
#download dataset from kaggle
path = Path('us-patent-phrase-to-phrase-matching')

##### `zipfile` is a module that provides tools to create, read, write, append, and list a ZIP file.


In [10]:
#use the kaggle api to downlaod the dataset to the above path and extract it

if not path.exists(): #if dataset not already present
  import zipfile,kaggle
  kaggle.api.competition_download_cli(str(path))
  zipfile.ZipFile(f'{path}.zip').extractall(path)


100%|██████████| 682k/682k [00:00<00:00, 66.3MB/s]

In [11]:
!ls {path} #we use {} in bash commands to use python variables

ls: cannot access '{path}': No such file or directory


Check out the data description at: https://www.kaggle.com/competitions/us-patent-phrase-to-phrase-matching/data

In [12]:
#creating a dataframe for the tarining data
df = pd.read_csv(path/'train.csv')  
df

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


In [13]:
df.describe(include = 'object')

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186


In [14]:
"""We are turning this in to a classification task by classifying into 3 categories like "related", "somewhat related", "not related" ,
we do this by making input something like TEXT1: abatement; TEXT2: eliminating process" """

df['input'] = 'TEXT_1: '  + df.context + '; TEXT_2: ' + df.target + ' ; ANC1: ' + df.anchor

In [15]:
df.input.head()

0    TEXT_1: A47; TEXT_2: abatement of pollution ; ...
1    TEXT_1: A47; TEXT_2: act of abating ; ANC1: ab...
2    TEXT_1: A47; TEXT_2: active catalyst ; ANC1: a...
3    TEXT_1: A47; TEXT_2: eliminating process ; ANC...
4    TEXT_1: A47; TEXT_2: forest region ; ANC1: aba...
Name: input, dtype: object

Transformers uses a Dataset object for storing a... well a dataset.



In [16]:
ds = Dataset.from_pandas(df) #from pandas dataframe to Dataset in Transformers

In [17]:
ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

To give this as input to the Deep Learning Model, we need to perform:
1) Tokenization : split each text up into words aka tokens.
2) Numericalization : Covert each token into a number.

Which technique to use to do above things, depends on the model we use.

In [18]:
model_name = 'microsoft/deberta-v3-small'

In [19]:
#we use Autotokenizer to create a tokenizer appropriate for the above model
import sentencepiece as spm
from transformers import AutoModelForSequenceClassification,AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.9/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [20]:
#looking at tokenizer's work

tokenizer.tokenize("this is an example but the, actual patent looks different; lets hope it works!")

['▁this',
 '▁is',
 '▁an',
 '▁example',
 '▁but',
 '▁the',
 ',',
 '▁actual',
 '▁patent',
 '▁looks',
 '▁different',
 ';',
 '▁lets',
 '▁hope',
 '▁it',
 '▁works',
 '!']

In [21]:
def tokenize_inp(x):
  
  """Function to tokenize our input"""
  return tokenizer(x['input'])

In [22]:
#we can use broadcasting to apply() tokenize_inp to the input: but that changes the input column in place

# ds['input'] = ds['input'].apply(tokenize_inp)

#OR
#we can use map() to create a new dataset with the modified column

tok_ds = ds.map(tokenize_inp,batched = True)

Map:   0%|          | 0/36473 [00:00<?, ? examples/s]

In [23]:
#This adds a new item to our dataset called input_ids. For instance, here is the input and IDs for the first row of our data:

#the ids here are linked to the vocabulary, represent the word's id in the vocab

row = tok_ds[0]
row['input'], row['input_ids']

('TEXT_1: A47; TEXT_2: abatement of pollution ; ANC1: abatement',
 [1,
  54453,
  616,
  435,
  294,
  336,
  5753,
  346,
  54453,
  616,
  445,
  294,
  47284,
  265,
  6435,
  2600,
  23702,
  435,
  294,
  47284,
  2])

In [24]:
v_d = tokenizer.get_vocab()
print(len(v_d.keys()))

128001


In [25]:
vaue  = [i for i in v_d.keys() if v_d[i] == 47284]
print(vaue)

['▁abatement']


In [26]:
#Rename column "score" to "labels" because transformers assume our dataset has a "labels" column

tok_ds = tok_ds.rename_columns({'score':'labels'})

In [27]:
tok_ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 36473
})

Test and Validation Sets

In [28]:
eval_df = pd.read_csv(path/'test.csv')
eval_df.describe()

,id,anchor,target,context
count,36,36,36,36
unique,36,34,36,29
top,4112d61851461f60,el display,inorganic photoconductor drum,G02
freq,1,2,1,3


# Validation set

In [ ]:
#to create validation set from our dataset

dds = tok_ds.train_test_spit(0.25,seed = 42)

In [51]:
dds['test']

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 9119
})

In [52]:
dds['train']


Dataset({
    features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 27354
})

In [39]:
#test_set : Notice it doesnt have the score column

eval_df['input'] = 'TEXT_1: '  + eval_df.context + '; TEXT_2: ' + eval_df.target + ' ; ANC1: ' + eval_df.anchor

eval_ds = Dataset.from_pandas(eval_df).map(tokenize_inp,batched = True)

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

In [53]:
eval_ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 36
})

# Training our Model

In [40]:
from transformers import TrainingArguments,Trainer


In [41]:
#Hyperparameters

batch_size = 128
epochs = 5
learning_rate = 8e-5

In [42]:
args = TrainingArguments('Outputs_patentscore',
                         learning_rate = learning_rate ,
                         warmup_ratio= 0.1,
                         lr_scheduler_type ='cosine',
                         fp16 = True,
                         evaluation_strategy="epoch",
                         per_device_train_batch_size = batch_size,
                         per_device_eval_batch_size = batch_size*2,
                         num_train_epochs = epochs,
                         weight_decay = 0.01,
                         report_to = 'none')

In [43]:
def corr(x,y): return np.corrcoef(x,y)[0][1]
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

In [54]:
#Use Trainer from transformer to create our model


model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels = 1)

trainer = Trainer(model,
                  args,
                  train_dataset = dds['train'],
                  eval_dataset = dds['test'],   #we are using dds['test'] as our validation set while training and the eval_ds our final test set
                  tokenizer = tokenizer,
                  compute_metrics = corr_d)
                                                           

Some weights of the model checkpoint at microsoft/deberta-v3-small were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.weight', 'mask_predictions.classifier.weight', 'mask_predictions.classifier.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.dense.bias']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

In [55]:
#train our model

trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.025492,0.797181
2,No log,0.023268,0.820000
3,0.030900,0.022269,0.832355
4,0.030900,0.021815,0.837260
5,0.012500,0.021789,0.838070


TrainOutput(global_step=1070, training_loss=0.020983337019091455, metrics={'train_runtime': 275.634, 'train_samples_per_second': 496.202, 'train_steps_per_second': 3.882, 'total_flos': 966672163237800.0, 'train_loss': 0.020983337019091455, 'epoch': 5.0})

In [56]:
#predictions


preds = trainer.predict(eval_ds).predictions.astype(float)
preds

array([[ 0.51660156],
       [ 0.58984375],
       [ 0.54833984],
       [ 0.30834961],
       [-0.04403687],
       [ 0.50488281],
       [ 0.53125   ],
       [-0.01850891],
       [ 0.31762695],
       [ 1.10449219],
       [ 0.21264648],
       [ 0.24865723],
       [ 0.74072266],
       [ 0.91552734],
       [ 0.77001953],
       [ 0.42797852],
       [ 0.3112793 ],
       [-0.04229736],
       [ 0.64697266],
       [ 0.43481445],
       [ 0.40405273],
       [ 0.22851562],
       [ 0.17956543],
       [ 0.25      ],
       [ 0.58349609],
       [-0.02391052],
       [-0.05001831],
       [-0.03131104],
       [-0.02981567],
       [ 0.6015625 ],
       [ 0.33447266],
       [ 0.03707886],
       [ 0.75146484],
       [ 0.45019531],
       [ 0.46337891],
       [ 0.22338867]])

In [57]:
#clipping values of predictions that are <0 and >1

preds = np.clip(preds,0,1)
preds

array([[0.51660156],
       [0.58984375],
       [0.54833984],
       [0.30834961],
       [0.        ],
       [0.50488281],
       [0.53125   ],
       [0.        ],
       [0.31762695],
       [1.        ],
       [0.21264648],
       [0.24865723],
       [0.74072266],
       [0.91552734],
       [0.77001953],
       [0.42797852],
       [0.3112793 ],
       [0.        ],
       [0.64697266],
       [0.43481445],
       [0.40405273],
       [0.22851562],
       [0.17956543],
       [0.25      ],
       [0.58349609],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.6015625 ],
       [0.33447266],
       [0.03707886],
       [0.75146484],
       [0.45019531],
       [0.46337891],
       [0.22338867]])

In [59]:
#we can check accuracy of our predictions with using the below correct prediction scores:
correct_scores = [[0.51],
[0.65],
[0.5],
[0.32],
[0.],
[0.52],
[0.52],
[0.07],
[0.28],
[1.],
[0.25],
[0.22],
[0.71],
[0.88],
[0.73],
[0.41],
[0.33],
[0.],
[0.69],
[0.35],
[0.4],
[0.25],
[0.12],
[0.27],
[0.56],
[0.],
[0.],
[0.],
[0.],
[0.59],
[0.29],
[0.03],
[0.74],
[0.57],
[0.46],
[0.21]]

In [73]:
def calculate_dist(x,y):

  mean_dist = sum([(x[i][0] - y[i][0])**2 if (x[i][0] - y[i][0])**2 != 0 else 0 for i in range(len(x))])/len(x)
  # print(mean_dist)

  return mean_dist

In [75]:

print(f'MSE for our predictions against correct ones: {calculate_dist(preds,correct_scores)}')

MSE for our predictions against correct ones: 0.0013989835693190493
